In [ ]:
### Check deleted files and removes them from s3

from core.utils import Tibanna
from core import ff_utils
from datetime import datetime
from tqdm import tqdm_notebook

# set enviroment and key/connection
env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
s3 = tibanna.s3
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

files = ff_utils.get_metadata('/search/?type=File&status=deleted', connection=ff)
len_files = len(files['@graph'])
print("Files to be deleted %s" % len(files['@graph']))

num_on_s3_raw = s3.size(s3.raw_file_bucket)
num_on_s3_processed = s3.size(s3.outfile_bucket)
print("Starting off with Raw count %d and Processed count %d on s3" %
       (num_on_s3_raw, num_on_s3_processed))


num_deleted_raw = 0
num_deleted_processed = 0
raw_files = 'FileFastq'
file_types_not_found = []


for i in tqdm_notebook(range(len_files)):
    the_file = files['@graph'][i]
    # double check cause I'm paranoid
    if the_file['status'].lower() == 'deleted':
        upload_key = the_file['upload_key']
        file_type = the_file['@type']
        try:
            if raw_files in file_type:
                s3.delete_key(upload_key, s3.raw_file_bucket)
                num_deleted_raw += 1
            else:
                s3.delete_key(upload_key, s3.outfile_bucket)
                num_deleted_processed += 1
                file_types_not_found.extend(file_type)
        
        except:
            print("failed to delete obj " + the_file['uuid'] + " with type " + file_type)

            
num_after_s3_raw = s3.size(s3.raw_file_bucket)
num_after_s3_processed = s3.size(s3.outfile_bucket)

print("raw deleted: %s" % num_deleted_raw)
print("processed deleted: %s" % num_deleted_processed)

print("s3 raw count: %s" % num_after_s3_raw)
print("s3 processed count: %s" % num_after_s3_processed)

s3_raw_var = num_on_s3_raw - num_after_s3_raw
s3_proc_var = num_on_s3_processed - num_after_s3_processed

raw_var = s3_raw_var - num_deleted_raw
proc_var = s3_proc_var - num_deleted_processed

print("raw variance (positive value mean more objects in s3): %d" % raw_var)
print("processed variance (positive value mean more objects in s3): %d" % proc_var)

unique_file_types = set(file_types_not_found)
print("Catch all file types %s" % unique_file_types)
